In [2]:
#!pip install rdflib
import rdflib
from rdflib import Graph
import pandas as pd
import json
import urllib.parse

In [ ]:
g = Graph()
g.parse("kg-mic.ttl")

<Graph identifier=Ndd1dc18bd4274f20b7279cab27f191b1 (<class 'rdflib.graph.Graph'>)>

In [16]:
print(len(g))

32284


In [17]:
df = pd.read_csv("../../output/X_test_true.tsv", delimiter = "\t")
df

,subject,CHEBI:100147,CHEBI:12931,CHEBI:12936,CHEBI:132943,CHEBI:132950,CHEBI:133748,CHEBI:13705,CHEBI:143136,CHEBI:14314,...,temp_range:very_low,temperature:mesophilic,temperature:psychrophilic,temperature:thermophilic,trophic_type:chemoheterotrophy,trophic_type:chemoorganoheterotrophy,trophic_type:chemoorganotrophy,trophic_type:chemotrophy,trophic_type:heterotrophy,trophic_type:organotrophy
0,NCBITaxon:931533,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,NCBITaxon:77585,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,NCBITaxon:1168289,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,NCBITaxon:728005,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,NCBITaxon:1423760,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,NCBITaxon:129921,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
403,NCBITaxon:442562,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
404,NCBITaxon:185294,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
405,NCBITaxon:1117320,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
pairs = []
for i in range(len(df)): 
    for j in range (1, len(df.columns)):
        if df.iloc[i, j] == 1:
            sub = df.iloc[i, 0]
            val = df.columns[j] 
            pairs.append([sub, val])

In [19]:
len(pairs)

8850

In [20]:
bioregistry_prefixes = {}
f = open("../../registry.json")
data = json.load(f)
for entity in data.values():
    bioregistry_prefixes[entity["prefix"]] = {"name": entity["name"], "uri_format": entity["uri_format"]}
f.close()
del data



def extract_uri(id: str):
    uri_parts = id.split(":", 1)
    if len(uri_parts) == 1:
        encoded_id = urllib.parse.quote(id, safe="")
        return f"urn:unknown:{encoded_id}"
    elif len(uri_parts) > 1:
        prefix = uri_parts[0].lower()
        def_prefix = bioregistry_prefixes.get(prefix)
        if def_prefix is None:
            encoded_id = urllib.parse.quote(id, safe="")
            return f"urn:unknown:{encoded_id}"
        else:
            uf = def_prefix["uri_format"]  # type: str
            if uf is None or "$1" not in uf:
                return f"https://bioregistry.io/{prefix}:{uri_parts[1]}"
            else:
                return uf.replace("$1", uri_parts[1])
    return None


pairs_prefixed = []
for row in pairs:
    uri_1 = extract_uri(row[0])
    uri_2 = extract_uri(row[1])
    pairs_prefixed.append([uri_1, uri_2])

len(pairs_prefixed)

8850

In [21]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
new_graph = Graph()

for row in pairs_prefixed:
    subjectitos = rdflib.term.URIRef(row[0])
    objectitos = rdflib.term.URIRef(row[1])
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos,
        "o": objectitos
    })
    for row in filtered_gq:
        new_graph.add(row)
        g.remove(row)

print(len(g), "  ", len(new_graph), "  ", len(g)+len(new_graph))


22566    9718    32284


In [22]:
df = pd.read_csv("../../0522/ytest.tsv", delimiter = "\t")
df

,subject,medium
0,NCBITaxon:931533,medium:11
1,NCBITaxon:77585,medium:830
2,NCBITaxon:1168289,medium:514
3,NCBITaxon:728005,medium:830
4,NCBITaxon:1423760,medium:11
...,...,...
402,NCBITaxon:129921,medium:65
403,NCBITaxon:442562,medium:830
404,NCBITaxon:185294,medium:693
405,NCBITaxon:1117320,medium:514


In [23]:
media_prefixed = []
for row in df.iterrows():
    uri_1 = extract_uri(row[1][0])
    uri_2 = "urn:unknown:medium%3A"+row[1][1].split(":")[1]
    media_prefixed.append([uri_1, uri_2])
print(len(media_prefixed))


407


C:\Users\Dominika\AppData\Local\Temp\ipykernel_7992\165415670.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  uri_1 = extract_uri(row[1][0])
C:\Users\Dominika\AppData\Local\Temp\ipykernel_7992\165415670.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  uri_2 = "urn:unknown:medium%3A"+row[1][1].split(":")[1]


In [24]:
media_prefixed

[['http://purl.obolibrary.org/obo/NCBITaxon_931533',
  'urn:unknown:medium%3A11'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_77585',
  'urn:unknown:medium%3A830'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_1168289',
  'urn:unknown:medium%3A514'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_728005',
  'urn:unknown:medium%3A830'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_1423760',
  'urn:unknown:medium%3A11'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_1120964',
  'urn:unknown:medium%3A514'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_344165',
  'urn:unknown:medium%3A830'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_2666047',
  'urn:unknown:medium%3A830'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_568098',
  'urn:unknown:medium%3A514'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_66854', 'urn:unknown:medium%3A65'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_1262554',
  'urn:unknown:medium%3A514'],
 ['http://purl.obolibrary.org/obo/NCBITaxon_184924',
  'urn:unknown:medium%3A5

In [25]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in media_prefixed:
    subjectitos = rdflib.term.URIRef(row[0])
    objectitos = rdflib.term.URIRef(row[1])
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos,
        "o": objectitos
    })
    for row in filtered_gq:
        new_graph.add(row)
        g.remove(row)

print(len(g), "  ", len(new_graph), "  ", len(g)+len(new_graph))


22159    10125    32284


In [26]:
new_graph.serialize(destination="original_test_true.ttl")

<Graph identifier=N25e8476d7b474ca6b9669ecce88da0a6 (<class 'rdflib.graph.Graph'>)>

In [41]:
g.serialize(destination="original_train.ttl")

<Graph identifier=Nd873ad879ca84fc3936662100722f934 (<class 'rdflib.graph.Graph'>)>

In [3]:
q = """
SELECT ?s ?p ?o
{
    ?s ?p ?o
}
"""

gq = g.query(q)


all_taxons = []

for row in gq:
    if (row.s[0:40] == "http://purl.obolibrary.org/obo/NCBITaxon"):
        all_taxons.append(rdflib.term.URIRef(row.s))

print(len(all_taxons))

3823031


In [4]:
len(all_taxons)
all_taxons = list(dict.fromkeys(all_taxons))
print(len(all_taxons))

815981


In [5]:
import random
new_train_taxons = random.sample(all_taxons,k=int(0.8*len(all_taxons)))

In [6]:
len(new_train_taxons)

652784

In [8]:
new_test_taxons = []
new_test_taxons = set(all_taxons).difference(new_train_taxons)

In [9]:
len(new_test_taxons)

163197

In [12]:
import csv
with open('new_train_taxons.csv', 'w', newline='\n') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerows([new_train_taxons])

In [13]:
import csv
with open('new_test_taxons.csv', 'w', newline='\n') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerows([new_test_taxons])

In [3]:
df = pd.read_csv("new_test_taxons.csv", delimiter = ",")


In [14]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
print(len(g))

for row in new_train_taxons:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos
    })
    for row in filtered_gq:
        g.remove(row)

print(len(g))



6827261
3769886


In [4]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
print(len(g))

for row in df:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos
    })
    for row in filtered_gq:
        g.remove(row)

print(len(g))



6827261
6061605


In [5]:
g.serialize(destination="new_trainset.ttl")

<Graph identifier=Nec10acaeb81a4327b52bb0bfe43409a0 (<class 'rdflib.graph.Graph'>)>

In [15]:
g.serialize(destination="new_testset.ttl")

<Graph identifier=N46c4dea75a8349218c8666a4c6eefd34 (<class 'rdflib.graph.Graph'>)>

In [3]:
#g.serialize(destination="kg-mic.ttl")

<Graph identifier=N39181cd92372482691fdbadb96595a2b (<class 'rdflib.graph.Graph'>)>

In [5]:
print(len(g))

6827261


In [6]:
df = pd.read_csv("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/test_data.tsv", delimiter = "\t")
df

,subject,CHEBI:100147,CHEBI:12931,CHEBI:12936,CHEBI:132943,CHEBI:132950,CHEBI:133748,CHEBI:13705,CHEBI:143136,CHEBI:14314,...,temp_range:very_low,temperature:mesophilic,temperature:psychrophilic,temperature:thermophilic,trophic_type:chemoheterotrophy,trophic_type:chemoorganoheterotrophy,trophic_type:chemoorganotrophy,trophic_type:chemotrophy,trophic_type:heterotrophy,trophic_type:organotrophy
0,NCBITaxon:1122600,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,NCBITaxon:684552,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,NCBITaxon:452922,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,NCBITaxon:447421,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,NCBITaxon:273384,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,NCBITaxon:1465756,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1227,NCBITaxon:872983,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1228,NCBITaxon:184924,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1229,NCBITaxon:1223526,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
taxons = []
for i in range(len(df)): 
    taxons.append(df.iloc[i, 0])

In [8]:
len(taxons)

1231

In [9]:

bioregistry_prefixes = {}
f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/registry.json")
data = json.load(f)
for entity in data.values():
    bioregistry_prefixes[entity["prefix"]] = {"name": entity["name"], "uri_format": entity["uri_format"]}
f.close()
del data



def extract_uri(id: str):
    uri_parts = id.split(":", 1)
    if len(uri_parts) == 1:
        encoded_id = urllib.parse.quote(id, safe="")
        return f"urn:unknown:{encoded_id}"
    elif len(uri_parts) > 1:
        prefix = uri_parts[0].lower()
        def_prefix = bioregistry_prefixes.get(prefix)
        if def_prefix is None:
            encoded_id = urllib.parse.quote(id, safe="")
            return f"urn:unknown:{encoded_id}"
        else:
            uf = def_prefix["uri_format"]  # type: str
            if uf is None or "$1" not in uf:
                return f"https://bioregistry.io/{prefix}:{uri_parts[1]}"
            else:
                return uf.replace("$1", uri_parts[1])
    return None


taxons_prefixed = []
for row in taxons:
    uri_1 = extract_uri(row)
    taxons_prefixed.append(uri_1)

len(taxons_prefixed)

1231

In [10]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
test_graph = Graph()

print(len(g))

for row in taxons_prefixed:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos
    })
    for row in filtered_gq:
        g.remove(row)
        test_graph.add(row)

print(len(g))
print(len(test_graph))
print(len(g)+len(test_graph))



6827261
6788212
39049
6827261


In [15]:
g.serialize(destination="train_data_1023.ttl")

<Graph identifier=N8cc5c99cce6a486fb8ceec1352db14f6 (<class 'rdflib.graph.Graph'>)>

In [16]:
test_graph.serialize(destination="test_data_1023.ttl")

<Graph identifier=Nfd03e068f5054d33b143afdfb15693ec (<class 'rdflib.graph.Graph'>)>

In [ ]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in taxons_prefixed:
    objectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "o": objectitos
    })
    new_graph += filtered_gq

print(len(new_graph))

In [9]:
new_real_graph = Graph()
for row in new_graph:
    new_real_graph.add(row)

len(new_real_graph)

39049

In [10]:
for row in new_real_graph:
    g.remove(row)

In [11]:
len(g)

6788212

In [12]:
g.serialize(destination="train_data_from_spaqrl_onlysub.ttl")

<Graph identifier=Naa1cef661b914ad49718bd46e2c01577 (<class 'rdflib.graph.Graph'>)>

In [2]:
g = Graph()

g.parse("train_data_from_spaqrl_onlysub.ttl")

KeyboardInterrupt: 

In [5]:
q = """
SELECT ?s ?p ?o
{
    ?s ?p ?o
}
"""

gq = g.query(q)


train_taxons = []

for row in gq:
    if (row.s[0:40] == "http://purl.obolibrary.org/obo/NCBITaxon"):
        train_taxons.append(rdflib.term.URIRef(row.s))

print(len(train_taxons))

3783982


In [6]:
import csv
with open('train_taxonst_second_try.tsv', 'w', newline='\n') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerows([train_taxons])

In [3]:
df = pd.read_csv("train_taxonst_second_try.tsv", lineterminator = ",", header=None)


In [4]:
df

,0
0,http://purl.obolibrary.org/obo/NCBITaxon_1241306
1,http://purl.obolibrary.org/obo/NCBITaxon_2719337
2,http://purl.obolibrary.org/obo/NCBITaxon_1004261
3,http://purl.obolibrary.org/obo/NCBITaxon_1633686
4,http://purl.obolibrary.org/obo/NCBITaxon_2644557
...,...
3783977,http://purl.obolibrary.org/obo/NCBITaxon_1051666
3783978,http://purl.obolibrary.org/obo/NCBITaxon_1250044
3783979,http://purl.obolibrary.org/obo/NCBITaxon_1467204
3783980,http://purl.obolibrary.org/obo/NCBITaxon_2804029


In [5]:
new_g = Graph()

new_g.parse("kg-mic.ttl")

<Graph identifier=Nfcf0a50203ae47ff82aa712fa6ce8a8c (<class 'rdflib.graph.Graph'>)>

In [6]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
print("zacatek: ",len(new_g))

for i in range(len(df)): 
    subjectitos = rdflib.term.URIRef(df[0][i])
    filtered_gq = new_g.query(query, initBindings = {
        "s": subjectitos
    })
    for row in filtered_gq:
        new_g.remove(row)
    if i % 100000 == 0:
        print(i)

print("konec: ",len(new_g))

zacatek:  6827261
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
konec:  3043279


In [7]:
new_g.serialize(destination="test_data_from_spaqrl_onlysub.ttl")

<Graph identifier=Nfcf0a50203ae47ff82aa712fa6ce8a8c (<class 'rdflib.graph.Graph'>)>

In [44]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
new_graph_train = Graph()

filtered_gq = g.query(query, initBindings = {
        "o": rdflib.term.URIRef("urn:unknown:medium%3A92")
    })

print(len(filtered_gq))

157


In [45]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

#new_graph_show = Graph()
for row in filtered_gq:
    filtered = g.query(query, initBindings = {
        "s": rdflib.term.URIRef(row.s)
    })
    new_graph_show += filtered

print(len(new_graph_show))

42839


In [46]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in filtered_gq:
    filtered = g.query(query, initBindings = {
        "o": rdflib.term.URIRef(row.s)
    })
    new_graph_show += filtered

print(len(new_graph_show))

44917


In [47]:
show_graph = Graph()
for row in new_graph_show:
    show_graph.add(row)

len(show_graph)

44917

In [48]:
show_graph.serialize(destination="bestof_graph.ttl")

<Graph identifier=N2e6f0ecb84e54dcf82d2825c5dd28301 (<class 'rdflib.graph.Graph'>)>